In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

In [ ]:
# Additional dependencies for this notebook
!pip install -q qiskit-addon-mpf

*Utilisation estimée du QPU : quatre minutes sur un processeur Heron r2 (REMARQUE : il s'agit d'une estimation uniquement. Votre temps d'exécution peut varier.)*

## Contexte

Ce tutoriel démontre comment utiliser une Formule Multi-Produit (MPF) pour obtenir une erreur de Trotter plus faible sur notre observable par rapport à celle encourue par le circuit de Trotter le plus profond que nous exécuterons réellement.
Les MPF réduisent l'erreur de Trotter de la dynamique hamiltonienne grâce à une combinaison pondérée de plusieurs exécutions de circuits. Considérez la tâche de trouver les valeurs d'espérance d'observable pour l'état quantique
$\rho(t)=e^{-i H t} \rho(0) e^{i H t}$ avec l'hamiltonien $H$. On peut utiliser les Formules de Produit (PF) pour approximer l'évolution temporelle $e^{-i H t}$ en procédant comme suit :

- Écrire l'hamiltonien $H$ comme $H=\sum_{a=1}^d F_a,$ où $F_a$ sont des opérateurs hermitiens tels que chaque unitaire correspondant puisse être implémenté efficacement sur un dispositif quantique.
- Approximer les termes $F_a$ qui ne commutent pas entre eux.

Ensuite, la PF de premier ordre (formule de Lie-Trotter) est :

$$S_1(t):=\prod_{a=1}^d e^{-i F_a t},$$

qui a un terme d'erreur quadratique $S_1(t)=e^{-i H t}+\mathcal{O}\left(t^{2}\right)$. On peut également utiliser des PF d'ordre supérieur (formules de Lie-Trotter-Suzuki), qui convergent plus rapidement, et qui sont définies récursivement comme :

$$S_2(t):=\prod_{a=1}^d e^{-i F_a t/2}\prod_{a=1}^d e^{-i F_a t/2}$$

$$S_{2 \chi}(t):= S_{2 \chi -2}(s_{\chi}t)^2 S_{2 \chi -2}((1-4s_{\chi})t)S_{2 \chi -2}(s_{\chi}t)^2,$$

où $\chi$ est l'ordre de la PF symétrique et $s_p = \left( 4 - 4^{1/(2p-1)} \right)^{-1}$. Pour les évolutions temporelles longues, on peut diviser l'intervalle de temps $t$ en $k$ intervalles, appelés pas de Trotter, de durée $t/k$ et approximer l'évolution temporelle dans chaque intervalle avec une formule de produit d'ordre $\chi$ $S_{\chi}$. Ainsi, la PF d'ordre $\chi$ pour l'opérateur d'évolution temporelle sur $k$ pas de Trotter est :

$$ S_{\chi}^{k}(t) = \left[ S_{\chi} \left( \frac{t}{k} \right)\right]^k = e^{-i H t}+O\left(t \left( \frac{t}{k} \right)^{\chi} \right)$$

où le terme d'erreur diminue avec le nombre de pas de Trotter $k$ et l'ordre $\chi$ de la PF.

Étant donné un entier $k \geq 1$ et une formule de produit $S_{\chi}(t)$, l'état approximatif évolué temporellement $\rho_k(t)$ peut être obtenu à partir de $\rho_0$ en appliquant $k$ itérations de la formule de produit $S_{\chi}\left(\frac{t}{k}\right)$.

$$
\rho_k(t)=S_{\chi}\left(\frac{t}{k}\right)^k \rho_0 S_{\chi}\left(\frac{t}{k}\right)^{-k}
$$

$\rho_k(t)$ est une approximation de $\rho(t)$ avec l'erreur d'approximation de Trotter ||$\rho_k(t)-\rho(t) ||$. Si nous considérons une combinaison linéaire d'approximations de Trotter de $\rho(t)$ :

$$
\mu(t) = \sum_{j}^{l} x_j \rho^{k_j}_{j}\left(\frac{t}{k_j}\right) + \text{une certaine erreur de Trotter résiduelle} \, ,
$$

où $x_j$ sont nos coefficients de pondération, $\rho^{k_j}_j$ est la matrice densité correspondant à l'état pur obtenu en faisant évoluer l'état initial avec la formule de produit, $S^{k_j}_{\chi}$, impliquant $k_j$ pas de Trotter, et $j \in {1, ..., l}$ indexe le nombre de PF qui composent la MPF. Tous les termes dans $\mu(t)$ utilisent la même formule de produit $S_{\chi}(t)$ comme base.
L'objectif est d'améliorer ||$\rho_k(t)-\rho(t) \|$ en trouvant $\mu(t)$ avec un $\|\mu(t)-\rho(t)\|$ encore plus faible.

* $\mu(t)$ n'a pas besoin d'être un état physique car $x_i$ n'a pas besoin d'être positif. L'objectif ici est de minimiser l'erreur dans la valeur d'espérance des observables et non de trouver un remplacement physique pour $\rho(t)$.
* $k_j$ détermine à la fois la profondeur du circuit et le niveau d'approximation de Trotter. Des valeurs plus petites de $k_j$ conduisent à des circuits plus courts, qui encourent moins d'erreurs de circuit mais seront une approximation moins précise de l'état désiré.

La clé ici est que l'erreur de Trotter résiduelle donnée par $\mu(t)$ est plus petite que l'erreur de Trotter que l'on obtiendrait en utilisant simplement la plus grande valeur de $k_j$.

Vous pouvez voir l'utilité de ceci sous deux perspectives :

1. Pour un budget fixe de pas de Trotter que vous pouvez exécuter, vous pouvez obtenir des résultats avec une erreur de Trotter qui est plus petite au total.
2. Étant donné un nombre cible de pas de Trotter qui est trop grand pour être exécuté, vous pouvez utiliser la MPF pour trouver une collection de circuits de profondeur inférieure à exécuter qui résulte en une erreur de Trotter similaire.

## Prérequis

Avant de commencer ce tutoriel, assurez-vous d'avoir les éléments suivants installés :

* Qiskit SDK v1.0 ou version ultérieure, avec le support de [visualisation](https://docs.quantum.ibm.com/api/qiskit/visualization)
* Qiskit Runtime v0.22 ou version ultérieure (`pip install qiskit-ibm-runtime`)
* Addons Qiskit MPF (`pip install qiskit_addon_mpf`)
* Addons Qiskit utils (`pip install qiskit_addon_utils`)
* Bibliothèque Quimb (`pip install quimb`)
* Bibliothèque Qiskit Quimb (`pip install qiskit-quimb`)
* Numpy v0.21 pour la compatibilité entre les packages (`pip install numpy==0.21`)

## Partie I. Exemple à petite échelle

### Explorer la stabilité de la MPF

Il n'y a pas de restriction évidente sur le choix du nombre de pas de Trotter $k_j$ qui composent l'état MPF $\mu(t)$. Cependant, ceux-ci doivent être choisis avec soin pour éviter les instabilités dans les valeurs d'espérance résultantes calculées à partir de $\mu(t)$. Une bonne règle générale est de définir le plus petit pas de Trotter $k_{\text{min}}$ de sorte que $t/k_{\text{min}} \lt 1$. Si vous souhaitez en savoir plus à ce sujet et sur la façon de choisir vos autres valeurs de $k_j$, consultez le guide [Comment choisir les pas de Trotter pour une MPF](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html).

Dans l'exemple ci-dessous, nous explorons la stabilité de la solution MPF en calculant la valeur d'espérance de la magnétisation pour une plage de temps en utilisant différents états évolués temporellement. Plus précisément, nous comparons les valeurs d'espérance calculées à partir de chacune des évolutions temporelles approximatives implémentées avec les pas de Trotter correspondants et les différents modèles MPF (coefficients statiques et dynamiques) avec les valeurs exactes de l'observable évolué temporellement. Tout d'abord, définissons les paramètres pour les formules de Trotter et les temps d'évolution

In [1]:
import numpy as np

mpf_trotter_steps = [1, 2, 4]
order = 2
symmetric = False

trotter_times = np.arange(0.5, 1.55, 0.1)
exact_evolution_times = np.arange(trotter_times[0], 1.55, 0.05)

Pour cet exemple, nous utiliserons l'état de Néel comme état initial $\vert \text{Neel} \rangle = \vert 0101...01 \rangle$ et le modèle de Heisenberg sur une ligne de 10 sites pour l'hamiltonien régissant l'évolution temporelle

$$
\hat{\mathcal{H}}_{Heis} = J \sum_{i=1}^{L-1} \left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ Z_i Z_{(i+1)} \right) \, ,
$$

où $J$ est la force de couplage pour les arêtes de plus proches voisins.

In [2]:
from qiskit.transpiler import CouplingMap
from rustworkx.visualization import graphviz_draw
from qiskit_addon_utils.problem_generators import generate_xyz_hamiltonian
import numpy as np


L = 10

# Generate some coupling map to use for this example
coupling_map = CouplingMap.from_line(L, bidirectional=False)
graphviz_draw(coupling_map.graph, method="circo")

# Get a qubit operator describing the Heisenberg field model
hamiltonian = generate_xyz_hamiltonian(
    coupling_map,
    coupling_constants=(1.0, 1.0, 1.0),
    ext_magnetic_field=(0.0, 0.0, 0.0),
)


print(hamiltonian)

SparsePauliOp(['IIIIIIIXXI', 'IIIIIIIYYI', 'IIIIIIIZZI', 'IIIIIXXIII', 'IIIIIYYIII', 'IIIIIZZIII', 'IIIXXIIIII', 'IIIYYIIIII', 'IIIZZIIIII', 'IXXIIIIIII', 'IYYIIIIIII', 'IZZIIIIIII', 'IIIIIIIIXX', 'IIIIIIIIYY', 'IIIIIIIIZZ', 'IIIIIIXXII', 'IIIIIIYYII', 'IIIIIIZZII', 'IIIIXXIIII', 'IIIIYYIIII', 'IIIIZZIIII', 'IIXXIIIIII', 'IIYYIIIIII', 'IIZZIIIIII', 'XXIIIIIIII', 'YYIIIIIIII', 'ZZIIIIIIII'],
              coeffs=[1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j,
 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j, 1.+0.j])


The observable that we will be measuring is magnetization on a pair of qubits in the middle of the chain.

In [ ]:
from qiskit.quantum_info import SparsePauliOp

observable = SparsePauliOp.from_sparse_list(
    [("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L
)
print(observable)

SparsePauliOp(['IIIIZZIIII'],
              coeffs=[1.+0.j])


L'observable que nous mesurerons est la magnétisation sur une paire de qubits au milieu de la chaîne.

In [4]:
from qiskit.circuit.library import XXPlusYYGate
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.optimization.collect_and_collapse import (
    CollectAndCollapse,
    collect_using_filter_function,
    collapse_to_operation,
)
from functools import partial


def filter_function(node):
    return node.op.name in {"rxx", "ryy"}


collect_function = partial(
    collect_using_filter_function,
    filter_function=filter_function,
    split_blocks=True,
    min_block_size=1,
)


def collapse_to_xx_plus_yy(block):
    param = 0.0
    for node in block.data:
        param += node.operation.params[0]
    return XXPlusYYGate(param)


collapse_function = partial(
    collapse_to_operation,
    collapse_function=collapse_to_xx_plus_yy,
)

pm = PassManager()
pm.append(CollectAndCollapse(collect_function, collapse_function))

Then we create the circuits implementing the approximate Trotter time-evolutions.

In [5]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


# Initial Neel state preparation
initial_state_circ = QuantumCircuit(L)
initial_state_circ.x([i for i in range(L) if i % 2 != 0])


all_circs = []
for total_time in trotter_times:
    mpf_trotter_circs = [
        generate_time_evolution_circuit(
            hamiltonian,
            time=total_time,
            synthesis=SuzukiTrotter(reps=num_steps, order=order),
        )
        for num_steps in mpf_trotter_steps
    ]

    mpf_trotter_circs = pm.run(
        mpf_trotter_circs
    )  # Collect XX and YY into XX + YY

    mpf_circuits = [
        initial_state_circ.compose(circuit) for circuit in mpf_trotter_circs
    ]
    all_circs.append(mpf_circuits)

In [6]:
mpf_circuits[-1].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/92dc20a7-0.avif" alt="Output of the previous code cell" />

Ensuite, nous créons les circuits implémentant les évolutions temporelles approximatives de Trotter.

In [24]:
from copy import deepcopy
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager


aer_sim = AerSimulator()
estimator = Estimator(mode=aer_sim)

mpf_expvals_all_times, mpf_stds_all_times = [], []
for t, mpf_circuits in zip(trotter_times, all_circs):
    mpf_expvals = []
    circuits = [deepcopy(circuit) for circuit in mpf_circuits]
    pm_sim = generate_preset_pass_manager(
        backend=aer_sim, optimization_level=3
    )
    isa_circuits = pm_sim.run(circuits)
    result = estimator.run(
        [(circuit, observable) for circuit in isa_circuits], precision=0.005
    ).result()
    mpf_expvals = [res.data.evs for res in result]
    mpf_stds = [res.data.stds for res in result]
    mpf_expvals_all_times.append(mpf_expvals)
    mpf_stds_all_times.append(mpf_stds)

We also calculate the exact expectation values for comparison.

In [ ]:
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

exact_expvals = []
for t in exact_evolution_times:
    # Exact expectation values
    exp_H = expm(-1j * t * hamiltonian.to_matrix())
    initial_state = Statevector(initial_state_circ).data
    time_evolved_state = exp_H @ initial_state

    exact_obs = (
        time_evolved_state.conj()
        @ observable.to_matrix()
        @ time_evolved_state
    ).real
    exact_expvals.append(exact_obs)

![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/92dc20a7-0.avif)

Ensuite, nous calculons les valeurs d'espérance évoluées temporellement à partir des circuits de Trotter.

In [9]:
from qiskit_addon_mpf.static import setup_static_lse

lse = setup_static_lse(mpf_trotter_steps, order=order, symmetric=symmetric)

Nous calculons également les valeurs d'espérance exactes pour comparaison.

In [10]:
lse.A

array([[1.      , 1.      , 1.      ],
       [1.      , 0.25    , 0.0625  ],
       [1.      , 0.125   , 0.015625]])

#### Coefficients MPF statiques
Les MPF statiques sont celles où les valeurs $x_j$ ne dépendent pas du temps d'évolution, $t$. Considérons la PF d'ordre $\chi = 1$ avec $k_j$ pas de Trotter, cela peut s'écrire comme :

$$ S_1^{k_j}\left( \frac{t}{k_j} \right)=e^{-i H t}+ \sum_{n=1}^{\infty} A_n \frac{t^{n+1}}{k_j^n}  $$

où $A_n$ sont des matrices qui dépendent des commutateurs des termes $F_a$ dans la décomposition de l'hamiltonien. Il est important de noter que $A_n$ eux-mêmes sont indépendants du temps et du nombre de pas de Trotter $k_j$. Par conséquent, il est possible d'annuler les termes d'erreur d'ordre inférieur contribuant à $\mu(t)$ avec un choix soigneux des poids $x_j$ de la combinaison linéaire. Pour annuler l'erreur de Trotter pour les premiers $l-1$ termes (ceux-ci donneront les plus grandes contributions car ils correspondent au plus petit nombre de pas de Trotter) dans l'expression pour $\mu(t)$, les coefficients $x_j$ doivent satisfaire les équations suivantes :

$$ \sum_{j=1}^l x_j = 1 $$
$$ \sum_{j=1}^{l-1} \frac{x_j}{k_j^{n}} = 0 $$

avec $n=0, ... l-2$. La première équation garantit qu'il n'y a pas de biais dans l'état construit $\mu(t)$, tandis que la deuxième équation assure l'annulation des erreurs de Trotter. Pour les PF d'ordre supérieur, la deuxième équation devient $ \sum_{j=1}^{l-1} \frac{x_j}{k_j^{\eta}} = 0 $ où $\eta = \chi + 2n$ pour les PF symétriques et $\eta = \chi + n$ sinon, avec $n=0, ..., l-2$. L'erreur résultante (Réf. [\[1\]](#references),[\[2\]](#references)) est alors

$$ \epsilon = \mathcal{O} \left( \frac{t^{l+1}}{k_1^l} \right).$$

Déterminer les coefficients MPF statiques pour un ensemble donné de valeurs $k_j$ revient à résoudre le système linéaire d'équations défini par les deux équations ci-dessus pour les variables $x_j$ : $Ax=b$. Où $x$ sont nos coefficients d'intérêt, $A$ est une matrice qui dépend de $k_j$ et du type de PF que nous utilisons ($S$), et $b$ est un vecteur de contraintes. Plus précisément :

$$A_{0,j} = 1 $$
$$A_{i>0,j} = k_{j}^{-(\chi + s(i-1))}$$
$$b_0 = 1$$
$$b_{i>0} = 0 $$

où $\chi$ est l'``order``, $s$ est $2$ si ``symmetric`` est ``True`` et $1$ sinon, $k_{j}$ sont les ``trotter_steps``, et $x$ sont les variables à résoudre. Les indices $i$ et $j$ commencent à $0$. Nous pouvons également visualiser cela sous forme matricielle :

$$
A =
\begin{bmatrix}
A_{0,0} & A_{0,1} & A_{0,2} & ... \\
A_{1,0} & A_{1,1} & A_{1,2} & ...  \\
A_{2,0} & A_{2,1} & A_{2,2} & ...  \\
... & ... & ... & ...
\end{bmatrix} =
\begin{bmatrix}
1 & 1 & 1 & ... \\
k_{0}^{-(\chi + s(1-1))} & k_{1}^{-(\chi + s(1-1))} & k_{2}^{-(\chi + s(1-1))} & ... \\
k_{0}^{-(\chi + s(2-1))} & k_{1}^{-(\chi + s(2-1))} & k_{2}^{-(\chi + s(2-1))} & ... \\
... & ... & ... & ...
\end{bmatrix}
$$

et

$$
b =
\begin{bmatrix}
b_{0} \\
b_{1} \\
b_{2}  \\
...
\end{bmatrix} =
\begin{bmatrix}
1 \\
0 \\
0  \\
...
\end{bmatrix}
$$

Pour plus de détails, consultez la documentation du Système Linéaire d'Équations ([LSE](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.LSE.html)).

Nous pouvons trouver une solution pour $x$ analytiquement comme $x = A^{-1}b$ ; voir par exemple les Réf. [\[1\]](#references) ou [\[2\]](#references).
Cependant, cette solution exacte peut être « mal conditionnée », résultant en de très grandes normes L1 de nos coefficients, $x$, ce qui peut conduire à de mauvaises performances de la MPF.
À la place, on peut également obtenir une solution approximative qui minimise la norme L1 de $x$ afin de tenter d'optimiser le comportement de la MPF.
##### Configurer le LSE
Maintenant que nous avons choisi nos valeurs $k_j$, nous devons d'abord construire le LSE, $Ax=b$ comme expliqué ci-dessus.
La matrice $A$ dépend non seulement de $k_j$ mais aussi de notre choix de PF, en particulier son _ordre_.
De plus, vous pouvez prendre en compte si la PF est symétrique ou non (voir [\[1\]](#references)) en définissant `symmetric=True/False`.
Cependant, cela n'est pas obligatoire, comme le montre la Réf. [\[2\]](#references).

In [11]:
lse.b

array([1., 0., 0.])

Travaillons sur les valeurs choisies ci-dessus pour construire la matrice $A$ et le vecteur $b$. Avec $j=0,1, 2$ pas de Trotter $k_j = [1, 2, 4]$, ordre $\chi = 2$ et choix de pas de Trotter non symétriques ($s=1$), nous avons que les éléments de matrice de $A$ sous la première ligne sont déterminés par l'expression $A_{i>0,j} = k_{j}^{-(2 + 1(i-1))}$, spécifiquement :

$$ A_{0,0} = A_{0,1} = A_{0,2} =  1 $$
$$ A_{1,j} = k_{j}^{-1}  \rightarrow A_{1,0} = \frac{1}{1^2}, \;, A_{1,1} = \frac{1}{2^2}, \;, A_{1,2} = \frac{1}{4^2}$$
$$ A_{2,j} = k_{j}^{-2}  \rightarrow A_{2,0} = \frac{1}{1^3}, \;, A_{2,1} = \frac{1}{2^3}, \;, A_{2,2} = \frac{1}{4^3}$$

ou sous forme matricielle :

$$
A =
\begin{bmatrix}
1 & 1 & 1\\
1 & \frac{1}{2^2} & \frac{1}{4^2}  \\
1 & \frac{1}{2^3} & \frac{1}{4^3}  \\
\end{bmatrix}
$$

Cela est possible de voir en inspectant l'objet `lse` :

In [12]:
mpf_coeffs = lse.solve()
print(
    f"The static coefficients associated with the ansatze are: {mpf_coeffs}"
)

The static coefficients associated with the ansatze are: [ 0.04761905 -0.57142857  1.52380952]


##### Optimize for $x$ using an exact model

Alternatively to computing $x=A^{-1}b$, you can also use [setup_exact_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_exact_model.html) to construct a [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) instance that uses the LSE as constraints and whose optimal solution will yield $x$.

In the next section, it will be clear why this interface exists.

In [13]:
from qiskit_addon_mpf.costs import setup_exact_problem

model_exact, coeffs_exact = setup_exact_problem(lse)
model_exact.solve()
print(coeffs_exact.value)

[ 0.04761905 -0.57142857  1.52380952]


Tandis que le vecteur de contraintes $b$ a les éléments suivants :
$$ b_{0} = 1 $$
$$ b_1 = b_2 = 0 $$

Ainsi,

$$
b =
\begin{bmatrix}
1 \\
0 \\
0
\end{bmatrix}
$$

Et de manière similaire dans `lse` :

In [14]:
print(
    "L1 norm of the exact coefficients:",
    np.linalg.norm(coeffs_exact.value, ord=1),
)  # ord specifies the norm. ord=1 is for L1

L1 norm of the exact coefficients: 2.1428571428556378


##### Optimize for $x$ using an approximate model

It might happen that the L1 norm for the chosen set of $k_j$ values is deemed too high.
If that is the case and you cannot choose a different set of $k_j$ values, you can use an approximate solution to the LSE instead of an exact one.

To do so, simply use [setup_approximate_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_approximate_model.html) to construct a different [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) instance, which constrains the L1-norm to a chosen threshold while minimizing the difference of $Ax$ and $b$.

In [16]:
from qiskit_addon_mpf.costs import setup_sum_of_squares_problem

model_approx, coeffs_approx = setup_sum_of_squares_problem(
    lse, max_l1_norm=1.5
)
model_approx.solve()
print(coeffs_approx.value)
print(
    "L1 norm of the approximate coefficients:",
    np.linalg.norm(coeffs_approx.value, ord=1),
)

[-1.10294118e-03 -2.48897059e-01  1.25000000e+00]
L1 norm of the approximate coefficients: 1.5


L'objet `lse` a des méthodes pour trouver les coefficients statiques $x_j$ satisfaisant le système d'équations.

In [17]:
from qiskit_addon_utils.slicing import slice_by_depth
from qiskit_addon_mpf.backends.tenpy_layers import LayerModel
from qiskit_addon_mpf.backends.tenpy_layers import LayerwiseEvolver
from functools import partial

# Create approximate time-evolution circuits
single_2nd_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=order)
)
single_2nd_order_circ = pm.run(single_2nd_order_circ)  # collect XX and YY

# Find layers in the circuit
layers = slice_by_depth(single_2nd_order_circ, max_slice_depth=1)

# Create tensor network models
models = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz") for layer in layers
]

# Create the time-evolution object
approx_factory = partial(
    LayerwiseEvolver,
    layers=models,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 2,
    },
)

<Admonition type="warning">
The options of `LayerwiseEvolver` that determine the details of the tensor network simulation must be chosen carefully to avoid setting up an ill-defined optimization problem.
</Admonition>

We then set up the exact evolver (for example, [`ExactEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ExactEvolverFactory)), which returns an [`Evolver`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.html#qiskit_addon_mpf.backends.Evolver) object computing the true or “reference” time-evolution. Realistically, we would approximate the exact evolution by using a higher-order Suzuki–Trotter formula or another reliable method with a small time step. Below, we approximate the exact time-evolved state with a fourth-order Suzuki-Trotter formula using a small time step `dt=0.1`, which means the number of Trotter steps used at time $t$ is $k=t/dt$. We also specify some TeNPy-specific truncation options to bound the maximum bond dimension of the underlying tensor network, as well as the minimum singular values of the split tensor network bonds. These parameters can affect the accuracy of the expectation value calculated with the dynamic MPF coefficients, so it is important to explore a range of values to find the optimal balance between computational time and accuracy. Note that the calculation of the MPF coefficients does not rely of the expectation value of the PF obtained from hardware execution, and therefore it can be tuned in post-processing.

In [20]:
single_4th_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=4)
)
single_4th_order_circ = pm.run(single_4th_order_circ)
exact_model_layers = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz")
    for layer in slice_by_depth(single_4th_order_circ, max_slice_depth=1)
]

exact_factory = partial(
    LayerwiseEvolver,
    layers=exact_model_layers,
    dt=0.1,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 2,
    },
)

##### Optimiser pour $x$ en utilisant un modèle exact
Alternativement au calcul de $x=A^{-1}b$, vous pouvez également utiliser [setup_exact_model](https://qiskit.github.io/qiskit-addon-mpf/stubs/qiskit_addon_mpf.static.setup_exact_model.html) pour construire une instance de [cvxpy.Problem](https://www.cvxpy.org/api_reference/cvxpy.problems.html#cvxpy.Problem) qui utilise le LSE comme contraintes et dont la solution optimale donnera $x$.

Dans la section suivante, il sera clair pourquoi cette interface existe.

In [ ]:
from qiskit_addon_mpf.backends.tenpy_tebd import MPOState
from qiskit_addon_mpf.backends.tenpy_tebd import MPS_neel_state


def identity_factory():
    return MPOState.initialize_from_lattice(models[0].lat, conserve=True)


mps_initial_state = MPS_neel_state(models[0].lat)

For each time step $t$ we set up the dynamic linear system of equations with the [`setup_dynamic_lse`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html) method. The corresponding object contains the information about the dynamic MPF problem: `lse.A` gives the Gram matrix $M$ while `lse.b` gives the overlap $L$. We can then solve the LSE (when not ill-defined) to find the dynamic coefficients using the `setup_frobenius_problem`. It is important to note the difference with the static coefficients, which only depend on the details of the product formula used and are independent of the details of the time-evolution (Hamiltonian and initial state).

In [22]:
from qiskit_addon_mpf.dynamic import setup_dynamic_lse
from qiskit_addon_mpf.costs import setup_frobenius_problem

mpf_dynamic_coeffs_list = []
for t in trotter_times:
    print(f"Computing dynamic coefficients for time={t}")
    lse = setup_dynamic_lse(
        mpf_trotter_steps,
        t,
        identity_factory,
        exact_factory,
        approx_factory,
        mps_initial_state,
    )
    problem, coeffs = setup_frobenius_problem(lse)
    try:
        problem.solve()
        mpf_dynamic_coeffs_list.append(coeffs.value)
    except Exception as error:
        mpf_dynamic_coeffs_list.append(np.zeros(len(mpf_trotter_steps)))
        print(error, "Calculation Failed for time", t)
    print("")

Computing dynamic coefficients for time=0.5

Computing dynamic coefficients for time=0.6

Computing dynamic coefficients for time=0.7

Computing dynamic coefficients for time=0.7999999999999999

Computing dynamic coefficients for time=0.8999999999999999

Computing dynamic coefficients for time=0.9999999999999999

Computing dynamic coefficients for time=1.0999999999999999

Computing dynamic coefficients for time=1.1999999999999997

Computing dynamic coefficients for time=1.2999999999999998

Computing dynamic coefficients for time=1.4

Computing dynamic coefficients for time=1.4999999999999998



Comme indicateur pour savoir si une MPF construite avec ces coefficients donnera de bons résultats, nous pouvons utiliser la norme L1 (voir également Réf. [\[1\]](#references)).

In [ ]:
import matplotlib.pyplot as plt

sym = {1: "^", 2: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    trotter_curve, trotter_curve_error = [], []
    for trotter_expvals, trotter_stds in zip(
        mpf_expvals_all_times, mpf_stds_all_times
    ):
        trotter_curve.append(trotter_expvals[k])
        trotter_curve_error.append(trotter_stds[k])

    plt.errorbar(
        trotter_times,
        trotter_curve,
        yerr=trotter_curve_error,
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )  # , , )

# Get expectation values at all times for the static MPF with exact coeffs
exact_mpf_curve, exact_mpf_curve_error = [], []
for trotter_expvals, trotter_stds in zip(
    mpf_expvals_all_times, mpf_stds_all_times
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(coeffs_exact.value, trotter_stds)
            ]
        )
    )
    exact_mpf_curve_error.append(mpf_std)
    exact_mpf_curve.append(trotter_expvals @ coeffs_exact.value)

plt.errorbar(
    trotter_times,
    exact_mpf_curve,
    yerr=exact_mpf_curve_error,
    markersize=4,
    marker="o",
    label="Static MPF - Exact",
    color="purple",
)


# Get expectation values at all times for the static MPF with approximate
approx_mpf_curve, approx_mpf_curve_error = [], []
for trotter_expvals, trotter_stds in zip(
    mpf_expvals_all_times, mpf_stds_all_times
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(coeffs_approx.value, trotter_stds)
            ]
        )
    )
    approx_mpf_curve_error.append(mpf_std)
    approx_mpf_curve.append(trotter_expvals @ coeffs_approx.value)

plt.errorbar(
    trotter_times,
    approx_mpf_curve,
    yerr=approx_mpf_curve_error,
    markersize=4,
    marker="o",
    color="orange",
    label="Static MPF - Approximate",
)

# # Get expectation values at all times for the dynamic MPF
dynamic_mpf_curve, dynamic_mpf_curve_error = [], []
for trotter_expvals, trotter_stds, dynamic_coeffs in zip(
    mpf_expvals_all_times, mpf_stds_all_times, mpf_dynamic_coeffs_list
):
    mpf_std = np.sqrt(
        sum(
            [
                (coeff**2) * (std**2)
                for coeff, std in zip(dynamic_coeffs, trotter_stds)
            ]
        )
    )
    dynamic_mpf_curve_error.append(mpf_std)
    dynamic_mpf_curve.append(trotter_expvals @ dynamic_coeffs)

plt.errorbar(
    trotter_times,
    dynamic_mpf_curve,
    yerr=dynamic_mpf_curve_error,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)


plt.plot(
    exact_evolution_times,
    exact_expvals,
    lw=3,
    color="red",
    label="Exact time-evolution",
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) as a function of time"
)
plt.xlabel("Time")
plt.ylabel("Expectation Value")
plt.legend()
plt.grid()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/2da9c948-0.avif" alt="Output of the previous code cell" />

In the cases like the example above, where the $k=1$ PF behaves poorly at all times, the quality of the dynamic MPF results is also heavily affected. In such situations, it is useful to investigate the possibility of using individual PFs with higher number of Trotter steps to improve the overall quality of the results. In these simulations, we see the interplay of different types of errors: error from finite sampling, and Trotter error from the product formulas. MPF helps to reduce the Trotter error due to the product formulas but incur in higher sampling error compared to the product formulas. This can be advantageous, as product formulas can reduce the sampling error with increased sampling, but the systematic error due to the Trotter approximation remains untouched.

Another interesting behavior that we can observe from the plot is that the expectation value for the PF for $k=1$ starts to behave erratically (on top of not being a good approximation for the exact one) at times for which $t/k > 1 $, as explained in the [guide](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html) on how to choose the number of Trotter steps.

### Step 1: Map classical inputs to a quantum problem
Let's now consider a single time $t=1.0$ and calculate the expectation value of the magnetization with the various methods using one QPU. The particular choice of $t$ was done so to maximize the difference between the various methods and observe their relative efficacy. To determine the window of time for which dynamic MPF is guaranteed to produce observables with lower error than any of the individual Trotter formulas within the multi-product, we can implement the “MPF test” - see equation (17) and surrounding text in [\[3\]](#references).

#### Set up the Trotter circuits

At this point, we have found our expansion coefficients, $x$, and all that is left to do is to generate the Trotterized quantum circuits.
Once again, the [qiskit_addon_utils.problem_generators](/docs/api/qiskit-addon-utils/problem-generators) module comes to the rescue with a useful function to do this:

In [26]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


total_time = 1.0
mpf_circuits = []
for k in mpf_trotter_steps:
    # Initial Neel state preparation
    circuit = QuantumCircuit(L)
    circuit.x([i for i in range(L) if i % 2 != 0])

    trotter_circ = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(order=order, reps=k),
        time=total_time,
    )

    circuit.compose(trotter_circ, inplace=True)

    mpf_circuits.append(pm.run(circuit))

In [27]:
mpf_circuits[-1].draw("mpl", fold=-1, scale=0.4)

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/87d2ac0c-0.avif" alt="Output of the previous code cell" />

### Step 2: Optimize problem for quantum hardware execution
Let's return to the calculation of the expectation value for a single time point. We'll pick a backend for executing the experiment on hardware.

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService


service = QiskitRuntimeService()
backend = service.least_busy(min_num_qubits=127)
print(backend)

qubits = list(range(backend.num_qubits))

Notez que vous avez une liberté complète sur la façon de résoudre ce problème d'optimisation, ce qui signifie que vous pouvez changer le solveur d'optimisation, ses seuils de convergence, etc.
Consultez le guide respectif sur [Comment utiliser le modèle approximatif.](https://qiskit.github.io/qiskit-addon-mpf/how_tos/using_approximate_model.html)
#### Coefficients MPF dynamiques
Dans la section précédente, nous avons introduit une MPF statique qui améliore l'approximation de Trotter standard. Cependant, cette version statique ne minimise pas nécessairement l'erreur d'approximation. Concrètement, la MPF statique, notée $\mu^S(t)$, n'est pas la projection optimale de $\rho(t)$ sur le sous-espace engendré par les états de formule de produit ${\rho_{k_i}(t)}_{i=1}^r$.

Pour remédier à cela, nous considérons une MPF dynamique (introduite dans la Réf. [\[2\]](#references) et démontrée expérimentalement dans la Réf. [\[3\]](#references)) qui minimise l'erreur d'approximation dans la norme de Frobenius. Formellement, nous nous concentrons sur la minimisation de

$$
\|\rho(t) - \mu^D(t)\|_F^2 \;=\; \mathrm{Tr}\bigl[ \left( \rho(t) - \mu^D(t)\right)^2 \bigr],
$$

par rapport à certains coefficients $x_i(t)$ à chaque temps $t$. Le projecteur *optimal* dans la norme de Frobenius est alors $\mu^D(t) \;=\; \sum_{i=1}^r x_i(t)\,\rho_{k_i}(t)$, et nous appelons $\mu^D(t)$ la MPF *dynamique*. En insérant les définitions ci-dessus :

$$
\|\rho(t) - \mu^D(t)\|_F^2
\;=\; \\
= \mathrm{Tr}\bigl[ \left( \rho(t) - \mu^D(t)\right)^2 \bigr]
\;=\; \\
= \mathrm{Tr}\bigl[ \left( \rho(t) - \sum_{i=1}^r x_i(t)\,\rho_{k_i}(t) \right) \left(  \rho(t) - \sum_{j=1}^r x_j(t)\,\rho_{k_j}(t) \right) \bigr]
\;=\; \\
= 1 \;+\; \sum_{i,j=1}^r M_{i,j}(t)\,x_i(t)\,x_j(t)
\;-\;
2 \sum_{i=1}^r L_i^{\mathrm{exact}}(t)\,x_i(t),
$$

où $M_{i,j}(t)$ est la *matrice de Gram*, définie par

$$
M_{i,j}(t) \;=\; \mathrm{Tr}\bigl[\rho_{k_i}(t)\,\rho_{k_j}(t)\bigr]
\;=\;
\bigl|\langle \psi_{\mathrm{in}} \!\mid S\bigl(t/k_i\bigr)^{-k_i}\,S\bigl(t/k_j\bigr)^{k_j} \!\mid \psi_{\mathrm{in}} \rangle \bigr|^2.
$$

et

$$
L_i^{\mathrm{exact}}(t) = \mathrm{Tr}[\rho(t)\,\rho_{k_i}(t)]
$$

représente le recouvrement entre l'état exact $\rho(t)$ et chaque approximation de formule de produit $\rho_{k_i}(t)$. Dans des scénarios pratiques, ces recouvrements peuvent seulement être mesurés approximativement en raison du bruit ou de l'accès partiel à $\rho(t)$.

Ici, $\lvert\psi_{\mathrm{in}}\rangle$ est l'état initial, et $S(\cdot)$ est l'opération appliquée dans la formule de produit. En choisissant les coefficients $x_i(t)$ qui minimisent cette expression (et en gérant les données de recouvrement approximatives lorsque $\rho(t)$ n'est pas entièrement connu), nous obtenons la « meilleure » (au sens de la norme de Frobenius) approximation dynamique de $\rho(t)$ dans le sous-espace MPF. Les quantités $L_i(t)$ et $M_{i,j}(t)$ peuvent être calculées efficacement en utilisant des méthodes de réseau de tenseurs [\[3\]](#references). L'addon Qiskit MPF fournit plusieurs « backends » pour effectuer le calcul. L'exemple ci-dessous montre la manière la plus flexible de le faire, et la documentation du [backend basé sur les couches TeNPy](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.tenpy_layers.html#module-qiskit_addon_mpf.backends.tenpy_layers) explique également en grand détail. Pour utiliser cette méthode, commencez par le circuit implémentant l'évolution temporelle souhaitée et créez des modèles qui représentent ces opérations à partir des couches du circuit correspondant. Enfin, un objet `Evolver` est créé qui peut être utilisé pour générer les quantités évoluées temporellement $M_{i,j}(t)$ et $L_i(t)$. Nous commençons par créer l'objet `Evolver` correspondant à l'évolution temporelle approximative ([`ApproxEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ApproxEvolverFactory)) implémentée par les circuits. En particulier, faites très attention à la variable `order` pour qu'elles correspondent. Notez que lors de la génération des circuits correspondant à l'évolution temporelle approximative, nous utilisons des valeurs de substitution pour le `time = 1.0` et le nombre de pas de Trotter (`reps=1`). Les circuits d'approximation corrects sont ensuite produits par le solveur de problème dynamique dans `setup_dynamic_lse`.

In [ ]:
import copy
from qiskit.transpiler import Target, CouplingMap

target = backend.target
instruction_2q = "cz"

cmap = target.build_coupling_map(filter_idle_qubits=True)
cmap_list = list(cmap.get_edges())

max_meas_err = 0.012
min_t2 = 40
max_twoq_err = 0.005

# Remove qubits with bad measurement or t2
cust_cmap_list = copy.deepcopy(cmap_list)
for q in range(target.num_qubits):
    meas_err = target["measure"][(q,)].error
    if target.qubit_properties[q].t2 is not None:
        t2 = target.qubit_properties[q].t2 * 1e6
    else:
        t2 = 0
    if meas_err > max_meas_err or t2 < min_t2:
        # print(q)
        for q_pair in cmap_list:
            if q in q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue

# Remove qubits with bad 2q gate or t2
for q in cmap_list:
    twoq_gate_err = target[instruction_2q][q].error
    if twoq_gate_err > max_twoq_err:
        # print(q)
        for q_pair in cmap_list:
            if q == q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue


cust_cmap = CouplingMap(cust_cmap_list)

cust_target = Target.from_configuration(
    basis_gates=backend.configuration().basis_gates
    + ["measure"],  # or whatever new set of gates
    coupling_map=cust_cmap,
)

sorted_components = sorted(
    [list(comp.physical_qubits) for comp in cust_cmap.connected_components()],
    reverse=True,
)
print("size of largest component", len(sorted_components[0]))

size of largest component 10


> **Warning:** Les options de `LayerwiseEvolver` qui déterminent les détails de la simulation de réseau de tenseurs doivent être choisies avec soin pour éviter de configurer un problème d'optimisation mal défini.
Nous configurons ensuite l'évoluteur exact (par exemple, [`ExactEvolverFactory`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.dynamic.html#qiskit_addon_mpf.dynamic.ExactEvolverFactory)), qui renvoie un objet [`Evolver`](https://qiskit.github.io/qiskit-addon-mpf/apidocs/qiskit_addon_mpf.backends.html#qiskit_addon_mpf.backends.Evolver) calculant l'évolution temporelle vraie ou « de référence ». De manière réaliste, nous approximerions l'évolution exacte en utilisant une formule de Suzuki-Trotter d'ordre supérieur ou une autre méthode fiable avec un petit pas de temps. Ci-dessous, nous approximons l'état évolué temporellement exact avec une formule de Suzuki-Trotter de quatrième ordre en utilisant un petit pas de temps `dt=0.1`, ce qui signifie que le nombre de pas de Trotter utilisés au temps $t$ est $k=t/dt$. Nous spécifions également certaines options de troncature spécifiques à TeNPy pour limiter la dimension de liaison maximale du réseau de tenseurs sous-jacent, ainsi que les valeurs singulières minimales des liaisons de réseau de tenseurs divisées. Ces paramètres peuvent affecter la précision de la valeur d'espérance calculée avec les coefficients MPF dynamiques, il est donc important d'explorer une plage de valeurs pour trouver l'équilibre optimal entre le temps de calcul et la précision. Notez que le calcul des coefficients MPF ne repose pas sur la valeur d'espérance de la PF obtenue à partir de l'exécution matérielle, et peut donc être ajusté en post-traitement.

In [34]:
cust_cmap.draw()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/c5d8e90b-0.avif" alt="Output of the previous code cell" />

Ensuite, créez l'état initial de votre système dans un format compatible avec TeNPy (par exemple, un `MPS_neel_state`=$\vert 0101...01 \rangle$). Cela configure la fonction d'onde à plusieurs corps que vous ferez évoluer dans le temps $\lvert\psi_{\mathrm{in}}\rangle$ comme un tenseur.

In [72]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

transpiler = generate_preset_pass_manager(
    optimization_level=3, target=cust_target
)

transpiled_circuits = [transpiler.run(circ) for circ in mpf_circuits]

qubits_layouts = [
    [
        idx
        for idx, qb in circuit.layout.initial_layout.get_physical_bits().items()
        if qb._register.name != "ancilla"
    ]
    for circuit in transpiled_circuits
]

transpiled_circuits = []
for circuit, layout in zip(mpf_circuits, qubits_layouts):
    transpiler = generate_preset_pass_manager(
        optimization_level=3, backend=backend, initial_layout=layout
    )
    transpiled_circuit = transpiler.run(circuit)
    transpiled_circuits.append(transpiled_circuit)


# transform the observable defined on virtual qubits to
# an observable defined on all physical qubits
isa_observables = [
    observable.apply_layout(circ.layout) for circ in transpiled_circuits
]

In [73]:
print(transpiled_circuits[-1].depth(lambda x: x.operation.num_qubits == 2))
print(transpiled_circuits[-1].count_ops())
transpiled_circuits[-1].draw("mpl", idle_wires=False, fold=False)

51
OrderedDict([('sx', 310), ('rz', 232), ('cz', 132), ('x', 19)])


<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/25ce07a6-1.avif" alt="Output of the previous code cell" />

### Step 3: Execute using Qiskit primitives
With the Estimator primitive we can obtain the estimation of expectation value from the QPU. We execute the optimized AQC circuits with additional error mitigation and suppression techniques.

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator


estimator = Estimator(mode=backend)
estimator.options.default_shots = 30000

# Set simple error suppression/mitigation options
estimator.options.dynamical_decoupling.enable = True
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.twirling.num_randomizations = "auto"
estimator.options.twirling.strategy = "active-accum"
estimator.options.resilience.measure_mitigation = True
estimator.options.experimental.execution_path = "gen3-turbo"

estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 3, 5)
estimator.options.resilience.zne.extrapolator = ("exponential", "linear")

estimator.options.environment.job_tags = ["mpf small"]


job = estimator.run(
    [
        (circ, observable)
        for circ, observable in zip(transpiled_circuits, isa_observables)
    ]
)

Enfin, tracez ces valeurs d'espérance tout au long du temps d'évolution.

In [39]:
result_exp = job.result()
evs_exp = [res.data.evs for res in result_exp]
evs_std = [res.data.stds for res in result_exp]

print(evs_exp)

[array(-0.06361607), array(-0.23820448), array(-0.50271805)]


![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/2da9c948-0.avif)

Dans les cas comme l'exemple ci-dessus, où la PF $k=1$ se comporte mal à tous les temps, la qualité des résultats MPF dynamiques est également fortement affectée. Dans de telles situations, il est utile d'étudier la possibilité d'utiliser des PF individuelles avec un nombre plus élevé de pas de Trotter pour améliorer la qualité globale des résultats. Dans ces simulations, nous voyons l'interaction de différents types d'erreurs : l'erreur due à l'échantillonnage fini, et l'erreur de Trotter due aux formules de produit. La MPF aide à réduire l'erreur de Trotter due aux formules de produit mais encourt une erreur d'échantillonnage plus élevée par rapport aux formules de produit. Cela peut être avantageux, car les formules de produit peuvent réduire l'erreur d'échantillonnage avec un échantillonnage accru, mais l'erreur systématique due à l'approximation de Trotter reste inchangée.

Un autre comportement intéressant que nous pouvons observer à partir du graphique est que la valeur d'espérance pour la PF pour $k=1$ commence à se comporter de manière erratique (en plus de ne pas être une bonne approximation pour l'exacte) aux temps pour lesquels $t/k > 1 $, comme expliqué dans le [guide](https://qiskit.github.io/qiskit-addon-mpf/how_tos/choose_trotter_steps.html) sur la façon de choisir le nombre de pas de Trotter.

### Étape 1 : Convertir les entrées classiques en un problème quantique
Considérons maintenant un temps unique $t=1.0$ et calculons la valeur d'espérance de la magnétisation avec les différentes méthodes en utilisant une seule QPU. Le choix particulier de $t$ a été fait afin de maximiser la différence entre les différentes méthodes et d'observer leur efficacité relative. Pour déterminer la fenêtre de temps pour laquelle la MPF dynamique est garantie de produire des observables avec une erreur plus faible que n'importe laquelle des formules de Trotter individuelles au sein du multi-produit, nous pouvons implémenter le "test MPF" - voir l'équation (17) et le texte environnant dans [\[3\]](#references).
#### Configurer les circuits de Trotter
À ce stade, nous avons trouvé nos coefficients d'expansion, $x$, et il ne reste plus qu'à générer les circuits quantiques trotterisés.
Une fois de plus, le module [qiskit_addon_utils.problem_generators](https://docs.quantum.ibm.com/api/qiskit-addon-utils/problem-generators) vient à la rescousse avec une fonction utile pour faire cela :

In [29]:
exact_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_exact.value, evs_std)
        ]
    )
)
print(
    "Exact static MPF expectation value: ",
    evs_exp @ coeffs_exact.value,
    "+-",
    exact_mpf_std,
)
approx_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_approx.value, evs_std)
        ]
    )
)
print(
    "Approximate static MPF expectation value: ",
    evs_exp @ coeffs_approx.value,
    "+-",
    approx_mpf_std,
)
dynamic_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(mpf_dynamic_coeffs_list[7], evs_std)
        ]
    )
)
print(
    "Dynamic MPF expectation value: ",
    evs_exp @ mpf_dynamic_coeffs_list[7],
    "+-",
    dynamic_mpf_std,
)

Exact static MPF expectation value:  -0.6329590442738475 +- 0.012798249760406036
Approximate static MPF expectation value:  -0.5690390035339492 +- 0.010459559917168473
Dynamic MPF expectation value:  -0.4655579758795695 +- 0.007639139186720507


Finally, for this small problem we can compute the exact reference value using [scipy.linalg.expm](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.expm.html) as follows:

In [30]:
from scipy.linalg import expm
from qiskit.quantum_info import Statevector

exp_H = expm(-1j * total_time * hamiltonian.to_matrix())

initial_state_circuit = QuantumCircuit(L)
initial_state_circuit.x([i for i in range(L) if i % 2 != 0])
initial_state = Statevector(initial_state_circuit).data

time_evolved_state = exp_H @ initial_state

exact_obs = (
    time_evolved_state.conj() @ observable.to_matrix() @ time_evolved_state
)
print("Exact expectation value ", exact_obs.real)

Exact expectation value  -0.39909900734489434


In [31]:
sym = {1: "^", 2: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    plt.errorbar(
        k,
        evs_exp[k],
        yerr=evs_std[k],
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )  # , , )


plt.errorbar(
    3,
    evs_exp @ coeffs_exact.value,
    yerr=exact_mpf_std,
    markersize=4,
    marker="o",
    color="purple",
    label="Static MPF",
)

plt.errorbar(
    4,
    evs_exp @ coeffs_approx.value,
    yerr=approx_mpf_std,
    markersize=4,
    marker="o",
    color="orange",
    label="Approximate static MPF",
)

plt.errorbar(
    5,
    evs_exp @ mpf_dynamic_coeffs_list[7],
    yerr=dynamic_mpf_std,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)

plt.axhline(
    y=exact_obs.real,
    linestyle="--",
    color="red",
    label="Exact time-evolution",
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) at time {total_time} for the different methods "
)
plt.xlabel("Method")
plt.ylabel("Expectation Value")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.grid(alpha=0.1)
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/a3eefe73-0.avif" alt="Output of the previous code cell" />

Ensuite, nous retirons les qubits aberrants de la carte de couplage pour nous assurer que l'étape de disposition du transpilateur ne les inclut pas. Ci-dessous, nous utilisons les propriétés du backend rapportées stockées dans l'objet `target` et retirons les qubits qui ont soit une erreur de mesure ou une porte à deux qubits au-dessus d'un certain seuil (`max_meas_err`, `max_twoq_err`), soit un temps $T_2$ (qui détermine la perte de cohérence) en dessous d'un certain seuil (`min_t2`).

In [32]:
def relative_error(ev, exact_ev):
    return abs(ev - exact_ev)


relative_error_k = [relative_error(ev, exact_obs.real) for ev in evs_exp]
relative_error_mpf = relative_error(evs_exp @ mpf_coeffs, exact_obs.real)
relative_error_approx_mpf = relative_error(
    evs_exp @ coeffs_approx.value, exact_obs.real
)
relative_error_dynamic_mpf = relative_error(
    evs_exp @ mpf_dynamic_coeffs_list[7], exact_obs.real
)

print("relative error for each trotter steps", relative_error_k)
print("relative error with MPF exact coeffs", relative_error_mpf)
print("relative error with MPF approx coeffs", relative_error_approx_mpf)
print("relative error with MPF dynamic coeffs", relative_error_dynamic_mpf)

relative error for each trotter steps [0.33548293650112293, 0.16089452939226306, 0.10361904247828346]
relative error with MPF exact coeffs 0.2338600369291003
relative error with MPF approx coeffs 0.16993999618905486
relative error with MPF dynamic coeffs 0.06645896853467514


## Part II: scale it up

Let's scale the problem up beyond what is possible to simulate exactly. In this section we will focus on reproducing some of the results shown in Ref. [\[3\]](#references).

### Step 1: Map classical inputs to a quantum problem

#### Hamiltonian

For the large-scale example, we use the XXZ model on a line of 50 sites:

$$
\hat{\mathcal{H}}_{XXZ} = \sum_{i=1}^{L-1} J_{i,(i+1)}\left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ 2\cdot Z_i Z_{(i+1)} \right) \, ,
$$

where $J_{i,(i+1)}$ is a random coefficient corresponding to edge $(i, i+1)$. This is the Hamiltonian considered in the demonstration presented in Ref. [\[3\]](#references).

In [33]:
L = 50
# Generate some coupling map to use for this example
coupling_map = CouplingMap.from_line(L, bidirectional=False)
graphviz_draw(coupling_map.graph, method="circo")

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/34bf68ac-0.avif" alt="Output of the previous code cell" />

In [ ]:
import numpy as np
from qiskit.quantum_info import SparsePauliOp, Pauli


# Generate random coefficients for our XXZ Hamiltonian
np.random.seed(0)
even_edges = list(coupling_map.get_edges())[::2]
odd_edges = list(coupling_map.get_edges())[1::2]

Js = np.random.uniform(0.5, 1.5, size=L)
hamiltonian = SparsePauliOp(Pauli("I" * L))
for i, edge in enumerate(even_edges + odd_edges):
    hamiltonian += SparsePauliOp.from_sparse_list(
        [
            ("XX", (edge), 2 * Js[i]),
            ("YY", (edge), 2 * Js[i]),
            ("ZZ", (edge), 4 * Js[i]),
        ],
        num_qubits=L,
    )

print(hamiltonian)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXX', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYY', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZ', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIXXIIIIIIIIII', 'IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIYYIIIIIIIIII', 'IIIIIIIIIIII

![Output of the previous code cell](../docs/images/tutorials/multi-product-formula/extracted-outputs/c5d8e90b-0.avif)

Nous pouvons ensuite mapper le circuit et l'observable sur les qubits physiques du dispositif.

In [35]:
observable = SparsePauliOp.from_sparse_list(
    [("ZZ", (L // 2 - 1, L // 2), 1.0)], num_qubits=L
)
print(observable)

SparsePauliOp(['IIIIIIIIIIIIIIIIIIIIIIIIZZIIIIIIIIIIIIIIIIIIIIIIII'],
              coeffs=[1.+0.j])


#### Choose Trotter steps
The experiment showcased in Fig. 4 of Ref. [\[3\]](#references) uses $k_j = [2, 3, 4]$ symmetric Trotter steps of order $2$. We focus on the results for time $t=3$, where the MPF and a PF with a higher number of Trotter steps (6 in this case) have the same Trotter error. However, the MPF expectation value is calculated from circuits corresponding to the lower number of Trotter steps and thus shallower. In practice, even if the MPF and the deeper Trotter steps circuit have the same Trotter error, we expect the experimental expectation value calculated from the MPF circuits to be closer to the theory one, as it entails running shallower circuits less exposed to hardware noise compared to the circuit corresponding to the higher Trotter step PF.

In [36]:
total_time = 3
mpf_trotter_steps = [2, 3, 4]
order = 2
symmetric = True

#### Set up the LSE
Here we look at the static MPF coefficients for this problem.

In [37]:
lse = setup_static_lse(mpf_trotter_steps, order=order, symmetric=symmetric)
mpf_coeffs = lse.solve()
print(
    f"The static coefficients associated with the ansatze are: {mpf_coeffs}"
)
print("L1 norm:", np.linalg.norm(mpf_coeffs, ord=1))

The static coefficients associated with the ansatze are: [ 0.26666667 -2.31428571  3.04761905]
L1 norm: 5.628571428571431


In [38]:
model_approx, coeffs_approx = setup_sum_of_squares_problem(
    lse, max_l1_norm=2.0
)
model_approx.solve()
print(coeffs_approx.value)
print(
    "L1 norm of the approximate coefficients:",
    np.linalg.norm(coeffs_approx.value, ord=1),
)

[-0.24255546 -0.25744454  1.5       ]
L1 norm of the approximate coefficients: 2.0


### Étape 4 : Post-traiter et renvoyer le résultat dans le format classique souhaité
La seule étape de post-traitement consiste à combiner la valeur d'espérance obtenue à partir des primitives Qiskit Runtime à différentes étapes de Trotter en utilisant les coefficients MPF respectifs. Pour un observable $A$ nous avons :

$$ \langle A \rangle_{\text{mpf}}  = \text{Tr} [A \mu(t)] = \sum_{j} x_j  \text{Tr} [A \rho_{k_j}] = \sum_{j} x_j \langle A \rangle_j$$

Tout d'abord, nous extrayons les valeurs d'espérance individuelles obtenues pour chacun des circuits de Trotter :

In [ ]:
# Create approximate time-evolution circuits
single_2nd_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=order)
)
single_2nd_order_circ = pm.run(single_2nd_order_circ)  # collect XX and YY

# Find layers in the circuit
layers = slice_by_depth(single_2nd_order_circ, max_slice_depth=1)

# Create tensor network models
models = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz") for layer in layers
]

# Create the time-evolution object
approx_factory = partial(
    LayerwiseEvolver,
    layers=models,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 4,
    },
)

# Create exact time-evolution circuits
single_4th_order_circ = generate_time_evolution_circuit(
    hamiltonian, time=1.0, synthesis=SuzukiTrotter(reps=1, order=4)
)
single_4th_order_circ = pm.run(single_4th_order_circ)
exact_model_layers = [
    LayerModel.from_quantum_circuit(layer, conserve="Sz")
    for layer in slice_by_depth(single_4th_order_circ, max_slice_depth=1)
]

# Create the time-evolution object
exact_factory = partial(
    LayerwiseEvolver,
    layers=exact_model_layers,
    dt=0.1,
    options={
        "preserve_norm": False,
        "trunc_params": {
            "chi_max": 64,
            "svd_min": 1e-8,
            "trunc_cut": None,
        },
        "max_delta_t": 3,
    },
)


def identity_factory():
    return MPOState.initialize_from_lattice(models[0].lat, conserve=True)


mps_initial_state = MPS_neel_state(models[0].lat)


lse = setup_dynamic_lse(
    mpf_trotter_steps,
    total_time,
    identity_factory,
    exact_factory,
    approx_factory,
    mps_initial_state,
)
problem, coeffs = setup_frobenius_problem(lse)
try:
    problem.solve()
    mpf_dynamic_coeffs = coeffs.value
except Exception as error:
    print(error, "Calculation Failed for time", total_time)
print("")

#### Construct each of the Trotter circuits in our MPF decomposition

In [41]:
from qiskit.synthesis import SuzukiTrotter
from qiskit_addon_utils.problem_generators import (
    generate_time_evolution_circuit,
)
from qiskit import QuantumCircuit


mpf_circuits = []
for k in mpf_trotter_steps:
    # Initial state preparation |1010..>
    circuit = QuantumCircuit(L)
    circuit.x([i for i in range(L) if i % 2])

    trotter_circ = generate_time_evolution_circuit(
        hamiltonian,
        synthesis=SuzukiTrotter(reps=k, order=order),
        time=total_time,
    )

    circuit.compose(trotter_circ, qubits=range(L), inplace=True)

    mpf_circuits.append(circuit)

Ensuite, nous les recombinons simplement avec nos coefficients MPF pour obtenir les valeurs d'espérance totales de la MPF. Ci-dessous, nous le faisons pour chacune des différentes manières par lesquelles nous avons calculé $x$.

In [42]:
k = 6

# Initial state preparation |1010..>
comp_circuit = QuantumCircuit(L)
comp_circuit.x([i for i in range(L) if i % 2])


trotter_circ = generate_time_evolution_circuit(
    hamiltonian,
    synthesis=SuzukiTrotter(reps=k, order=order),
    time=total_time,
)

comp_circuit.compose(trotter_circ, qubits=range(L), inplace=True)


mpf_circuits.append(comp_circuit)

### Step 2: Optimize problem for quantum hardware execution

In [ ]:
import copy
from qiskit.transpiler import Target, CouplingMap

target = backend.target
instruction_2q = "cz"

cmap = target.build_coupling_map(filter_idle_qubits=True)
cmap_list = list(cmap.get_edges())

max_meas_err = 0.055
min_t2 = 30
max_twoq_err = 0.01

# Remove qubits with bad measurement or t2
cust_cmap_list = copy.deepcopy(cmap_list)
for q in range(target.num_qubits):
    meas_err = target["measure"][(q,)].error
    if target.qubit_properties[q].t2 is not None:
        t2 = target.qubit_properties[q].t2 * 1e6
    else:
        t2 = 0
    if meas_err > max_meas_err or t2 < min_t2:
        # print(q)
        for q_pair in cmap_list:
            if q in q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue

# Remove qubits with bad 2q gate or t2
for q in cmap_list:
    twoq_gate_err = target[instruction_2q][q].error
    if twoq_gate_err > max_twoq_err:
        # print(q)
        for q_pair in cmap_list:
            if q == q_pair:
                try:
                    cust_cmap_list.remove(q_pair)
                except ValueError:
                    continue


cust_cmap = CouplingMap(cust_cmap_list)

cust_target = Target.from_configuration(
    basis_gates=backend.configuration().basis_gates
    + ["measure"],  # or whatever new set of gates
    coupling_map=cust_cmap,
)

sorted_components = sorted(
    [list(comp.physical_qubits) for comp in cust_cmap.connected_components()],
    reverse=True,
)
print("size of largest component", len(sorted_components[0]))

size of largest component 73


In [104]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

transpiler = generate_preset_pass_manager(
    optimization_level=3, target=cust_target
)

transpiled_circuits = [transpiler.run(circ) for circ in mpf_circuits]

qubits_layouts = [
    [
        idx
        for idx, qb in circuit.layout.initial_layout.get_physical_bits().items()
        if qb._register.name != "ancilla"
    ]
    for circuit in transpiled_circuits
]

transpiled_circuits = []
for circuit, layout in zip(mpf_circuits, qubits_layouts):
    transpiler = generate_preset_pass_manager(
        optimization_level=3, backend=backend, initial_layout=layout
    )
    transpiled_circuit = transpiler.run(circuit)
    transpiled_circuits.append(transpiled_circuit)


# transform the observable defined on virtual qubits to
# an observable defined on all physical qubits
isa_observables = [
    observable.apply_layout(circ.layout) for circ in transpiled_circuits
]

### Step 3: Execute using Qiskit primitives

In [ ]:
from qiskit_ibm_runtime import EstimatorV2 as Estimator

estimator = Estimator(mode=backend)
estimator.options.default_shots = 30000

# Set simple error suppression/mitigation options
estimator.options.dynamical_decoupling.enable = True
estimator.options.twirling.enable_gates = True
estimator.options.twirling.enable_measure = True
estimator.options.twirling.num_randomizations = "auto"
estimator.options.twirling.strategy = "active-accum"
estimator.options.resilience.measure_mitigation = True
estimator.options.experimental.execution_path = "gen3-turbo"

estimator.options.resilience.zne_mitigation = True
estimator.options.resilience.zne.noise_factors = (1, 1.2, 1.4)
estimator.options.resilience.zne.extrapolator = "linear"

estimator.options.environment.job_tags = ["mpf large"]

job_50 = estimator.run(
    [
        (circ, observable)
        for circ, observable in zip(transpiled_circuits, isa_observables)
    ]
)

### Step 4: Post-process and return result in desired classical format

In [46]:
result = job_50.result()
evs = [res.data.evs for res in result]
std = [res.data.stds for res in result]

print(evs)
print(std)

[array(-0.08034071), array(-0.00605026), array(-0.15345759), array(-0.18127293)]
[array(0.04482517), array(0.03438413), array(0.21540776), array(0.21520829)]


In [52]:
exact_mpf_std = np.sqrt(
    sum([(coeff**2) * (std**2) for coeff, std in zip(mpf_coeffs, std[:3])])
)
print(
    "Exact static MPF expectation value: ",
    evs[:3] @ mpf_coeffs,
    "+-",
    exact_mpf_std,
)
approx_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(coeffs_approx.value, std[:3])
        ]
    )
)
print(
    "Approximate static MPF expectation value: ",
    evs[:3] @ coeffs_approx.value,
    "+-",
    approx_mpf_std,
)
dynamic_mpf_std = np.sqrt(
    sum(
        [
            (coeff**2) * (std**2)
            for coeff, std in zip(mpf_dynamic_coeffs, std[:3])
        ]
    )
)
print(
    "Dynamic MPF expectation value: ",
    evs[:3] @ mpf_dynamic_coeffs,
    "+-",
    dynamic_mpf_std,
)

Exact static MPF expectation value:  -0.47510243192011536 +- 0.6613940032465087
Approximate static MPF expectation value:  -0.20914170384216998 +- 0.32341567460419135
Dynamic MPF expectation value:  -0.07994951978722761 +- 0.07423091963310202


In [ ]:
sym = {2: "^", 3: "s", 4: "p"}
# Get expectation values at all times for each Trotter step
for k, step in enumerate(mpf_trotter_steps):
    plt.errorbar(
        k,
        evs[k],
        yerr=std[k],
        alpha=0.5,
        markersize=4,
        marker=sym[step],
        color="grey",
        label=f"{mpf_trotter_steps[k]} Trotter steps",
    )


plt.errorbar(
    3,
    evs[-1],
    yerr=std[-1],
    alpha=0.5,
    markersize=8,
    marker="x",
    color="blue",
    label="6 Trotter steps",
)


plt.errorbar(
    4,
    evs[:3] @ mpf_coeffs,
    yerr=exact_mpf_std,
    markersize=4,
    marker="o",
    color="purple",
    label="Static MPF",
)

plt.errorbar(
    5,
    evs[:3] @ coeffs_approx.value,
    yerr=approx_mpf_std,
    markersize=4,
    marker="o",
    color="orange",
    label="Approximate static MPF",
)

plt.errorbar(
    6,
    evs[:3] @ mpf_dynamic_coeffs,
    yerr=dynamic_mpf_std,
    markersize=4,
    marker="o",
    color="pink",
    label="Dynamic MPF",
)

exact_obs = -0.24384471447172074  # Calculated via Tensor Network calculation
plt.axhline(
    y=exact_obs, linestyle="--", color="red", label="Exact time-evolution"
)


plt.title(
    f"Expectation values for (ZZ,{(L//2-1, L//2)}) at time {total_time} for the different methods "
)
plt.xlabel("Method")
plt.ylabel("Expectation Value")
plt.legend(loc="upper center", bbox_to_anchor=(0.5, -0.2), ncol=2)
plt.grid(alpha=0.1)
plt.tight_layout()
plt.show()

<Image src="../docs/images/tutorials/multi-product-formula/extracted-outputs/d751af7c-0.avif" alt="Output of the previous code cell" />

## Partie II : passage à l'échelle
Passons maintenant à un problème d'une échelle supérieure, au-delà de ce qui peut être simulé de manière exacte. Dans cette section, nous nous concentrerons sur la reproduction de certains résultats présentés dans la Réf. [\[3\]](#references).
### Étape 1 : Transformer les entrées classiques en un problème quantique
#### Hamiltonien
Pour l'exemple à grande échelle, nous utilisons le modèle XXZ sur une ligne de 50 sites :

$$
\hat{\mathcal{H}}_{XXZ} = \sum_{i=1}^{L-1} J_{i,(i+1)}\left(X_i X_{(i+1)}+Y_i Y_{(i+1)}+ 2\cdot Z_i Z_{(i+1)} \right) \, ,
$$

où $J_{i,(i+1)}$ est un coefficient aléatoire correspondant à l'arête $(i, i+1)$. Il s'agit de l'hamiltonien considéré dans la démonstration présentée dans la Réf. [\[3\]](#references).